In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/Colab Notebooks/standard/titanic

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/standard/titanic


In [9]:
import pandas as pd
import numpy as np

####################################
# 1) データ読み込み用
####################################
def load_data(train_path, test_path=None):
    """
    任意のCSV読み込み用。必要があればtrain/test両方を返す。
    """
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path) if test_path is not None else None
    return train_df, test_df

####################################
# 2) 基本情報の確認
####################################
def show_basic_info(df, df_name="DataFrame"):
    """
    - データの先頭行、サイズ、info()等を表示
    - カラム一覧・データ型・メモリ使用量の概要も info() で確認
    """
    print(f"\n====== Basic Info: {df_name} ======")
    print(f"Shape: {df.shape}")  # 行数・列数
    print("\n--- Head(5) ---")
    print(df.head(5))
    print("\n--- Info ---")
    print(df.info())

####################################
# 3) 欠損値の確認
####################################
def check_missing_values(df, df_name="DataFrame"):
    """
    - 欠損値の総数、カラムごとの欠損値数を表示
    """
    print(f"\n====== Missing Values: {df_name} ======")
    total_missing = df.isnull().sum().sum()
    print(f"Total missing values: {total_missing}")
    missing_per_col = df.isnull().sum()
    missing_per_col = missing_per_col[missing_per_col > 0]
    if not missing_per_col.empty:
        print("\n--- Missing per column ---")
        print(missing_per_col)
    else:
        print("No missing values.")

####################################
# 4) 重複データの確認
####################################
def check_duplicates(df, df_name="DataFrame"):
    """
    - 重複行の数を表示。不要な場合は削除検討。
    """
    print(f"\n====== Duplicates Check: {df_name} ======")
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        print(f"Found {dup_count} duplicate rows.")
    else:
        print("No duplicate rows found.")

####################################
# 5) カラムごとのユニーク数・カーディナリティ確認
####################################
def check_unique_values(df, df_name="DataFrame", top_n=5):
    """
    - 各カラムのユニーク値数を表示
    - カーディナリティが高いカテゴリ列などを把握するのに役立つ
    - さらに上位N件の値サンプルを表示
    """
    print(f"\n====== Unique Values (Top {top_n} samples): {df_name} ======")
    for col in df.columns:
        unique_count = df[col].nunique(dropna=False)
        print(f"{col}: {unique_count} unique values")
        # 上位N件のユニーク値をテキストで確認したい場合:
        # (数値カラムだと意味が薄いが、カテゴリ列などは参考になる)
        value_sample = df[col].drop_duplicates().head(top_n).values
        print(f"  Sample: {value_sample}")

####################################
# 6) 基本統計量の確認 (数値/カテゴリ)
####################################
def show_descriptive_stats(df, numeric_cols=None, categorical_cols=None, df_name="DataFrame"):
    """
    数値列・カテゴリ列で分けて describe() や value_counts() などを表示。
    """
    print(f"\n====== Descriptive Stats: {df_name} ======")
    # 数値列
    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if numeric_cols:
        print(f"\n--- Numeric Columns: {numeric_cols} ---")
        print(df[numeric_cols].describe())
    else:
        print("\nNo numeric columns found.")

    # カテゴリ列
    if categorical_cols is None:
        cat_dtypes = ["object", "category"]
        categorical_cols = [col for col in df.columns if df[col].dtype.name in cat_dtypes]
    if categorical_cols:
        print(f"\n--- Categorical Columns: {categorical_cols} ---")
        for col in categorical_cols:
            print(f"\nValue Counts: {col}")
            print(df[col].value_counts(dropna=False))
    else:
        print("\nNo categorical columns found.")

####################################
# 7) 歪度(Skewness)・尖度(Kurtosis) の確認
####################################
def check_skew_kurtosis(df, numeric_cols=None, df_name="DataFrame"):
    """
    数値列に対して、歪度(skew)・尖度(kurtosis)を計算して一覧表示。
    - |skew|が大きい(>1等)と分布が強く歪んでいる
    - kurtosisが大きいと裾が重い分布を示す
    """
    print(f"\n====== Skewness & Kurtosis: {df_name} ======")
    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if not numeric_cols:
        print("No numeric columns found.")
        return
    stats = []
    for col in numeric_cols:
        col_skew = df[col].skew()
        col_kurt = df[col].kurt()
        stats.append((col, col_skew, col_kurt))
    # 表示
    print("Column         Skewness       Kurtosis")
    for (c, s, k) in stats:
        print(f"{c:<14} {s:>10.4f} {k:>10.4f}")

####################################
# 8) 外れ値の簡易チェック (Zスコア or IQR)
####################################
def check_outliers(df, numeric_cols=None, method='zscore', threshold=3.0, df_name="DataFrame"):
    """
    - method='zscore' の場合、Zスコアの絶対値が threshold を超えるデータ数をカウント
    - method='iqr' の場合、1.5倍IQRを超えるデータ数をカウント
    ※あくまで目安。実際の外れ値処理は手動確認推奨。
    """
    print(f"\n====== Outlier Check ({method}): {df_name} ======")
    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if not numeric_cols:
        print("No numeric columns found.")
        return

    if method == 'zscore':
        # Zスコア計算: (x - mean) / std
        df_zscore = (df[numeric_cols] - df[numeric_cols].mean()) / df[numeric_cols].std()
        outlier_count = (df_zscore.abs() > threshold).sum()
        for col in numeric_cols:
            if outlier_count[col] > 0:
                print(f"{col}: {outlier_count[col]} outliers (threshold={threshold})")
    elif method == 'iqr':
        Q1 = df[numeric_cols].quantile(0.25)
        Q3 = df[numeric_cols].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        for col in numeric_cols:
            lb = lower_bound[col]
            ub = upper_bound[col]
            mask_outliers = (df[col] < lb) | (df[col] > ub)
            out_count = mask_outliers.sum()
            if out_count > 0:
                print(f"{col}: {out_count} outliers (IQR method)")
    else:
        print("Unknown method. Use 'zscore' or 'iqr'.")

####################################
# 9) 相関行列の表示
####################################
def check_correlations(df, numeric_cols=None, df_name="DataFrame"):
    """
    数値列同士の相関係数を表示。
    """
    print(f"\n====== Correlation Matrix: {df_name} ======")
    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_cols) > 1:
        corr_matrix = df[numeric_cols].corr()
        print(corr_matrix)
    else:
        print("Not enough numeric columns for correlation.")

####################################
# 10) ターゲット列との相関・統計など (回帰/分類 タスク前提)
####################################
def check_target_relation(df, target_col, numeric_cols=None, df_name="DataFrame"):
    """
    目的変数が既にわかっている場合に、その変数との相関を出すなどの一元的な確認。
    - 数値ターゲットなら相関を算出
    - カテゴリターゲットならvalue_countsやカテゴリ別の平均をチェック、など
    """
    print(f"\n====== Target Relation Check: {df_name} (Target={target_col}) ======")
    if target_col not in df.columns:
        print(f"Target column '{target_col}' not found in {df_name}.")
        return
    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # カラムが数値かどうか判断
    if np.issubdtype(df[target_col].dtype, np.number):
        # 数値ターゲット: 相関を表示
        if len(numeric_cols) > 1:
            corr_with_target = df[numeric_cols].corrwith(df[target_col]).sort_values(ascending=False)
            print("\n--- Correlation with target (descending) ---")
            print(corr_with_target)
        else:
            print("Not enough numeric columns to check correlation.")
    else:
        # カテゴリターゲット: 分布確認
        print(f"\n--- Category Target: {target_col} Value Counts ---")
        print(df[target_col].value_counts(dropna=False))

####################################
# メイン関数例
####################################
def main():
    # === 1) データ読み込み (パスは各自で変更) ===
    train_df, test_df = load_data('/content/drive/MyDrive/Colab Notebooks/standard/titanic/data/train.csv', '/content/drive/MyDrive/Colab Notebooks/standard/titanic/data/test.csv')

    # === 2) 基本情報の確認 ===
    show_basic_info(train_df, df_name="Train")
    if test_df is not None:
        show_basic_info(test_df, df_name="Test")

    # === 3) 欠損値の確認 ===
    check_missing_values(train_df, df_name="Train")
    if test_df is not None:
        check_missing_values(test_df, df_name="Test")

    # === 4) 重複データの確認 ===
    check_duplicates(train_df, df_name="Train")
    if test_df is not None:
        check_duplicates(test_df, df_name="Test")

    # === 5) ユニーク値・カーディナリティ確認 ===
    check_unique_values(train_df, df_name="Train", top_n=5)
    if test_df is not None:
        check_unique_values(test_df, df_name="Test", top_n=5)

    # === 6) 基本統計量の確認 ===
    show_descriptive_stats(train_df, df_name="Train")
    if test_df is not None:
        show_descriptive_stats(test_df, df_name="Test")

    # === 7) 歪度・尖度の確認 ===
    check_skew_kurtosis(train_df, df_name="Train")
    if test_df is not None:
        check_skew_kurtosis(test_df, df_name="Test")

    # === 8) 外れ値の簡易チェック ===
    check_outliers(train_df, method='zscore', threshold=3.0, df_name="Train")
    # check_outliers(train_df, method='iqr', df_name="Train")
    if test_df is not None:
        check_outliers(test_df, method='zscore', df_name="Test")

    # === 9) 相関行列の表示 ===
    check_correlations(train_df, df_name="Train")
    if test_df is not None:
        check_correlations(test_df, df_name="Test")

    # === 10) ターゲット列がある場合の確認 (例: 'SalePrice', 'Survived' etc) ===
    # データセットが回帰か分類かによってここを使い分け。
    # 例: 回帰であれば 'SalePrice' (数値)
    # 例: 分類であれば 'Survived' (0/1)
    target_col = 'SalePrice'  # ここは例。該当しないならスキップ
    check_target_relation(train_df, target_col, df_name="Train")

    # --- 他にも集計や深堀りしたい場合は各自追加 ---

if __name__ == "__main__":
    main()



====== Basic Info: Train ======
Shape: (891, 12)

--- Head(5) ---
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123